In [1]:
from sklearn import datasets,model_selection
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
nltk.download('perluniprops')
from nltk.tokenize.moses import MosesDetokenizer
import sklearn
import os
import pandas as pd
from sklearn import model_selection,metrics,naive_bayes,pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Format the files so we will extract info easier from them

In [0]:
allFiles=[]
for f in os.listdir('.'):
  if(os.path.isfile(f) and f[0]!='.'):
    new=True
    for i in range(len(allFiles)):
      if(allFiles[i][0][:-5]==f[:-5]):
        new=False
        allFiles[i].append(f)
    if(new):
      allFiles.append([f])

In [0]:
print(len(allFiles))

30


In [0]:
X=[]
Y=[]
sentences=[]
for i in range(len(allFiles)):
  sentences=[]
  for j in range(len(allFiles[i])):
    sentences.append([])
    filepath = allFiles[i][j]
    with open(filepath) as fp:  
      line = fp.readline()
      while line:
        sentences[j].append(line)
        line = fp.readline()
  for k in range(len(sentences[0])):
    if(sentences[0][k][0]!='#'):
      term1=sentences[0][k][:4]
      term2=sentences[1][k][:4]
      term3=sentences[2][k][:4]
      if(term1==term2 or term1==term3):
        X.append(sentences[0][k][4:])
        #print(term1,sentences[0][k][4:])
        Y.append(term1)
      elif(term2==term3):
        X.append(sentences[0][k][4:])
        #print(term2,sentences[0][k][4:])
        Y.append(term2)
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, random_state = 22,stratify=Y)


In [0]:
print(len(X),len(Y))
print(len(x_train),len(y_train))
print(len(x_test),len(y_test))
print(len(x_test[0]))

1035 1035
776 776
259 259
77


In [0]:
def prossesing (lemma,stopword,stemming,sentences):
  for i in range(len(sentences)):
    detokenizer = MosesDetokenizer()
    token_sentences=nltk.word_tokenize(sentences[i])
    if(stopword):
      token_sentences=[word for word in token_sentences if word not in stopwords.words('english')]
    if(stemming):
      stemmer = SnowballStemmer("english",ignore_stopwords=True)
      token_sentences=[stemmer.stem(word) for word in token_sentences]
    if(lemma):
      p=1
    sentences[i]=detokenizer.detokenize(token_sentences, return_str=True)
  return sentences


x_train_prossesed = prossesing(lemma=False,stopword=True,stemming=True,sentences=x_train)
x_test_prossesed = prossesing(lemma=False,stopword=False,stemming=True,sentences=x_test)

In [0]:
for i in range(len(x_train)):
  print(len(x_train_prossesed[i]),len(x_train[i]),x_train[i])

In [0]:

multiNB_pipe = pipeline.Pipeline([
     ('tfidf', TfidfVectorizer()),
     ('clf', naive_bayes.MultinomialNB())
])

# Parameters to explore for Multinomial Naive Bayes Classifier
params = {
    'tfidf__max_features': (5000, 10000, 2500),
    'tfidf__ngram_range': ((1, 1), (2, 2)),
    'tfidf__use_idf': (True, False),
    'clf__alpha': (0.05, 0.01,0.1),
}

# Find the best parameters for both feature extraction and classifier
grid_search_MNB = GridSearchCV(multiNB_pipe, params, cv=3, n_jobs=-1, verbose=1, return_train_score=True)

# Fit the data into the GridSearch
scores = cross_val_score(grid_search_MNB, x_train_prossesed, y_train, cv=5)
grid_search_MNB.fit(x_train_prossesed, y_train)

In [0]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(grid_search_MNB.best_estimator_)

Accuracy: 0.70 (+/- 0.06)
Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))])


In [0]:
best_pipe_MNB = grid_search_MNB.best_estimator_
print(grid_search_MNB.best_score_)

y_predicted=best_pipe_MNB.predict(x_test)
prec = metrics.precision_score(y_test,y_predicted.tolist(),average='weighted')
rec = metrics.recall_score(y_test,y_predicted,average='weighted')
f1 = metrics.f1_score(y_test,y_predicted,average='weighted')
print('Results: \n')
print('Precision: {}'.format(prec))
print('Recall: {}'.format(rec))
print('F1 Score: {}'.format(f1))


0.7061855670103093
['MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'OWNX', 'OWNX', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'AIMX', 'MISC', 'OWNX', 'CONT', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'AIMX', 'OWNX', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'OWNX', 'OWNX', 'OWNX', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'OWNX', 'OWNX', 'MISC', 'MISC', 'OWNX', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MIS

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
LogisticReg_pipe = pipeline.Pipeline([
     ('tfidf', TfidfVectorizer()),
     ('clf', LogisticRegression())
])

# Parameters to explore for Multinomial Naive Bayes Classifier
params = {
    'tfidf__max_features': (5000, 10000, 2500),
    'tfidf__ngram_range': ((1, 1), (2, 2)),
    'tfidf__use_idf': (True, False),
}

# Find the best parameters for both feature extraction and classifier
grid_search_LR = GridSearchCV(LogisticReg_pipe, params, cv=3, n_jobs=-1, verbose=1, return_train_score=True)

# Fit the data into the GridSearch
grid_search_LR.fit(x_train_prossesed, y_train)
best_pipe_LR=grid_search_LR.best_estimator_

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    1.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
best_pipe_LR = grid_search_LR.best_estimator_
print(grid_search_LR.best_score_)

y_predicted=best_pipe_LR.predict(x_test_prossesed)

count=0
for i in range(len(y_test)):
  if(y_predicted[i]==y_test[i]):
    count=count+1
print(count/len(y_test))
#print(y_predicted.tolist())
#print(y_test)

prec = metrics.precision_score(y_test,y_predicted,average='weighted')
rec = metrics.recall_score(y_test,y_predicted,average='weighted')
f1 = metrics.f1_score(y_test,y_predicted,average='weighted')
print('Results: \n')
print('Precision: {}'.format(prec))
print('Recall: {}'.format(rec))
print('F1 Score: {}'.format(f1))

0.6469072164948454
0.694980694980695
Results: 

Precision: 0.623760852460404
Recall: 0.694980694980695
F1 Score: 0.626595235921303


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
